PONIZEJ 3:1/1:3


In [ ]:
import pymongo
import numpy as np
import time
import json

def connect_mongo_and_return_collections(login, password, IP, port, database_name, collections_name):
    """
    Nawiązuje połączenie z bazą danych MongoDB i zwraca kolekcję do operacji.
    """
    conn = pymongo.MongoClient(f'mongodb://{login}:{password}@{IP}:{port}/')
    db = conn[f'{database_name}']
    matches_coll = db[f'{collections_name}']

    try:
        # Test połączenia
        conn.admin.command('ping')
        print("Połączenie z MongoDB nawiązane!")
    except Exception as e:
        print(f"Nie udało się połączyć z MongoDB: {e}")
        exit()

    return matches_coll


def get_all_matches():
    """
    Pobiera dane z kolekcji `matches` dla wyników 3:1 lub 1:3, wykluczając określone klucze.
    """
    pipeline = [
        {
            "$match": {
                "$and": [
                    {
                        "$or": [
                            { "ft_result": "3 : 1" },
                            { "ft_result": "1 : 3" }
                        ]
                    },
                    { "delta_elo": { "$ne": 0 } }
                ]
            }
        },
        {
            "$project": {
                "_id": 0,
                "odds_url": 0,
                "home_id": 0,
                "odd_home": 0,
                "odd_away": 0
            }
        },
        {
            "$limit": 100000
        }
    ]

    matches_coll = connect_mongo_and_return_collections(
        'root', 'NameSecret', 'IP_sECRET', '27017',
        'czech_liga_pro_test', 'matches'
    )

    start_time = time.time()
    all_matches = list(matches_coll.aggregate(pipeline))
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Czas wykonania zapytania: {execution_time:.6f} sekund")

    if not all_matches:
        return "Brak wyników zapytania."
    else:
        return all_matches


def calculate_actuals_by_sets(match_data):
    """
    Oblicza wartość actual_home i actual_away dla meczu o wyniku 3:1 lub 1:3.
    """
    if int(match_data['winner']) == 1:
        actual_home = 1
    else:
        actual_home = 0

    return actual_home, 1 - actual_home


def calculate_predicted_delta_elo(match_data, K):
    """
    Oblicza przewidywaną zmianę Elo na podstawie aktualnych wartości Elo i faktycznego wyniku.
    """
    expected_home = 1 / (1 + 10 ** ((match_data['away_elo'] - match_data['home_elo']) / 108))
    expected_away = 1 - expected_home

    actual_home, actual_away = calculate_actuals_by_sets(match_data)
    
    new_home_elo = K * (actual_home - expected_home)
    new_away_elo = K * (actual_away - expected_away)

    return 1.197 * (new_home_elo), new_away_elo


def load_corrections(json_path='corrections.json'):
    """
    Wczytuje dane korekcyjne z pliku JSON.
    Oczekiwany format: { "1.3": 0.2, "1.4": 0.3, ... }
    """
    with open(json_path, 'r') as f:
        corrections = json.load(f)
    return corrections


def adjust_predicted_delta(p, corrections):
    """
    Dopasowuje przewidywaną zmianę elo (p) na podstawie wczytanych z pliku JSON korekt.

    Zasada:
    1. Zaokrąglamy p do jednej cyfry po przecinku.
    2. Konwertujemy na string, żeby dopasować do klucza w słowniku.
    3. Jeśli jest w corrections, to znaczy że mamy dla tej wartości specjalną korektę.
       W tym wypadku 'korekta' może oznaczać nową wartość predicted_delta_elo. 
       Można zinterpretować to tak, że wartość w 'corrections' to finalna korekta (np. średnia delta_elo z terminala).
    4. Jeśli klucza nie ma w corrections, zostawiamy p bez zmian (lub można zrobić inną logikę).

    Zwracamy zaokrągloną do jednej cyfry po przecinku wartość.
    """

    p_rounded = round(p, 1)
    p_str = f"{p_rounded}"

    if p_str in corrections:
        adj = corrections[p_str]
    elif abs(p) >= 11.3: 
        # to ze nie bylo takiego przypadku w bazie danych nie znaczy ze nie moze sie wydarzyc. Zabezpieczenie w razie rzadkiego meczu.
        adj = p_rounded * 1.06
    else:
        # Jeśli nie ma dopasowania w słowniku korekt, zostawiamy p_rounded bez zmian.
        adj = p_rounded

    adj = round(adj, 1)
    return adj


def calculate_mean_error(matches, adjusted_predicted_deltas):
    """
    Funkcja oblicza średni błąd pomiędzy rzeczywistą zmianą elo (delta_elo),
    a zmodyfikowaną przewidywaną zmianą elo.
    """
    mean_error = 0
    for match_data, adj_pred in zip(matches, adjusted_predicted_deltas):
        mean_error += abs(match_data['delta_elo'] - adj_pred)

    if len(matches) > 0:
        return mean_error / len(matches)
    else:
        return 0


def main():
    matches = get_all_matches()

    if isinstance(matches, str):
        print(matches)
        return

    corrections = load_corrections('corrections_3_1.json')

    predicted_deltas = []
    mean_error = 0
    print("original_delta_elo | predicted_delta_elo | predicted_delta_elo_adjusted")
    for match_data in matches:
        pred = (calculate_predicted_delta_elo(match_data, 10))[0]
        predicted_deltas.append(pred)
        mean_error += abs(match_data['delta_elo'] - pred)

    if len(matches) > 0:
        print(f"Średni błąd: {mean_error/len(matches)}")
    else:
        print("Brak danych do obliczeń.")
        return

    adjusted_predicted_deltas = [adjust_predicted_delta(p, corrections) for p in predicted_deltas]

    adjusted_mean_error = calculate_mean_error(matches, adjusted_predicted_deltas)
    print(f"Średni błąd po modyfikacji predykcji: {adjusted_mean_error}")


    for val in np.arange(8.1, 12, 0.1):
        delta_elo_values = []
        for match_data, original_pred, adj_pred in zip(matches, predicted_deltas, adjusted_predicted_deltas):
            # Zaokrąglamy do jednej cyfry po przecinku
            if round(original_pred, 1) == round(val, 1):
                delta_elo_values.append(match_data['delta_elo'])
        
        if len(delta_elo_values) > 0:
            avg_delta_elo = sum(delta_elo_values) / len(delta_elo_values)
            print(f"Średnia delta_elo dla meczów, gdzie predicted_delta_elo == {val}: {avg_delta_elo}")
        else:
            print(f"Brak meczów o predicted_delta_elo == {val}")

    for match_data, original_pred, adj_pred in zip(matches, predicted_deltas, adjusted_predicted_deltas):
        print(f"{match_data['delta_elo']} | {round(original_pred,1)} | {adj_pred}")


if __name__ == "__main__":
    main()


Ponizej 3:2/2:3

In [ ]:
import pymongo
import numpy as np
import time
import json

def connect_mongo_and_return_collections(login, password, IP, port, database_name, collections_name):
    """
    Nawiązuje połączenie z bazą danych MongoDB i zwraca kolekcję do operacji.
    """
    conn = pymongo.MongoClient(f'mongodb://{login}:{password}@{IP}:{port}/')
    db = conn[f'{database_name}']
    matches_coll = db[f'{collections_name}']

    try:
        # Test połączenia
        conn.admin.command('ping')
        print("Połączenie z MongoDB nawiązane!")
    except Exception as e:
        print(f"Nie udało się połączyć z MongoDB: {e}")
        exit()

    return matches_coll


def get_all_matches():
    """
    Pobiera dane z kolekcji `matches` dla wyników 3:1 lub 1:3, wykluczając określone klucze.
    """
    pipeline = [
        {
            "$match": {
                "$and": [
                    {
                        "$or": [
                            { "ft_result": "3 : 2" },
                            { "ft_result": "2 : 3" }
                        ]
                    },
                    { "delta_elo": { "$ne": 0 } }
                ]
            }
        },
        {
            "$project": {
                "_id": 0,
                "odds_url": 0,
                "home_id": 0,
                "odd_home": 0,
                "odd_away": 0
            }
        },
        {
            "$limit": 100000
        }
    ]

    matches_coll = connect_mongo_and_return_collections(
        'root', "NAME_secret", 'IP_Secret', '27017',
        'czech_liga_pro_test', 'matches'
    )

    start_time = time.time()
    all_matches = list(matches_coll.aggregate(pipeline))
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Czas wykonania zapytania: {execution_time:.6f} sekund")

    if not all_matches:
        return "Brak wyników zapytania."
    else:
        return all_matches


def calculate_actuals_by_sets(match_data):
    """
    Oblicza wartość actual_home i actual_away dla meczu o wyniku 3:2 lub 2:3.
    """
    if int(match_data['winner']) == 1:
        actual_home = 1
    else:
        actual_home = 0

    return actual_home, 1 - actual_home


def calculate_predicted_delta_elo(match_data, K):
    """
    Oblicza przewidywaną zmianę Elo na podstawie aktualnych wartości Elo i faktycznego wyniku.
    """
    expected_home = 1 / (1 + 10 ** ((match_data['away_elo'] - match_data['home_elo']) / 108))
    expected_away = 1 - expected_home

    actual_home, actual_away = calculate_actuals_by_sets(match_data)
    
    new_home_elo = K * (actual_home - expected_home)
    new_away_elo = K * (actual_away - expected_away)

    return 0.8017 * (new_home_elo), new_away_elo


def load_corrections(json_path='corrections.json'):
    """
    Wczytuje dane korekcyjne z pliku JSON.
    Oczekiwany format: { "1.3": 0.2, "1.4": 0.3, ... }
    """
    with open(json_path, 'r') as f:
        corrections = json.load(f)
    return corrections


def adjust_predicted_delta(p, corrections):
    """
    Dopasowuje przewidywaną zmianę elo (p) na podstawie wczytanych z pliku JSON korekt.

    Zasada:
    1. Zaokrąglamy p do jednej cyfry po przecinku.
    2. Konwertujemy na string, żeby dopasować do klucza w słowniku.
    3. Jeśli jest w corrections, to znaczy że mamy dla tej wartości specjalną korektę.
       W tym wypadku 'korekta' może oznaczać nową wartość predicted_delta_elo. 
       Można zinterpretować to tak, że wartość w 'corrections' to finalna korekta (np. średnia delta_elo z terminala).
    4. Jeśli klucza nie ma w corrections, zostawiamy p bez zmian (lub można zrobić inną logikę).

    Zwracamy zaokrągloną do jednej cyfry po przecinku wartość.
    """

    p_rounded = round(p, 1)
    p_str = f"{p_rounded}"

    if p_str in corrections:
        adj = corrections[p_str]
    elif abs(p) >= 7.7: 
        # to ze nie bylo takiego przypadku w bazie danych nie znaczy ze nie moze sie wydarzyc. Zabezpieczenie w razie rzadkiego meczu.
        adj = p_rounded * 1.04
    else:
        # Jeśli nie ma dopasowania w słowniku korekt, zostawiamy p_rounded bez zmian.
        adj = p_rounded

    adj = round(adj, 1)
    return adj


def calculate_mean_error(matches, adjusted_predicted_deltas):
    """
    Funkcja oblicza średni błąd pomiędzy rzeczywistą zmianą elo (delta_elo),
    a zmodyfikowaną przewidywaną zmianą elo.
    """
    mean_error = 0
    for match_data, adj_pred in zip(matches, adjusted_predicted_deltas):
        mean_error += abs(match_data['delta_elo'] - adj_pred)

    if len(matches) > 0:
        return mean_error / len(matches)
    else:
        return 0


def main():
    matches = get_all_matches()

    if isinstance(matches, str):
        print(matches)
        return

    corrections = load_corrections('corrections_3_2.json')

    predicted_deltas = []
    mean_error = 0
    print("original_delta_elo | predicted_delta_elo | predicted_delta_elo_adjusted")
    for match_data in matches:
        pred = (calculate_predicted_delta_elo(match_data, 10))[0]
        predicted_deltas.append(pred)
        mean_error += abs(match_data['delta_elo'] - pred)

    if len(matches) > 0:
        print(f"Średni błąd: {mean_error/len(matches)}")
    else:
        print("Brak danych do obliczeń.")
        return

    adjusted_predicted_deltas = [adjust_predicted_delta(p, corrections) for p in predicted_deltas]

    adjusted_mean_error = calculate_mean_error(matches, adjusted_predicted_deltas)
    print(f"Średni błąd po modyfikacji predykcji: {adjusted_mean_error}")


    for val in np.arange(5.5, 9, 0.1):
        delta_elo_values = []
        for match_data, original_pred, adj_pred in zip(matches, predicted_deltas, adjusted_predicted_deltas):
            # Zaokrąglamy do jednej cyfry po przecinku
            if round(original_pred, 1) == round(val, 1):
                delta_elo_values.append(match_data['delta_elo'])
        
        if len(delta_elo_values) > 0:
            avg_delta_elo = sum(delta_elo_values) / len(delta_elo_values)
            print(f"{val}: {avg_delta_elo}")
        else:
            print(f"Brak meczów o predicted_delta_elo == {val}")

    for match_data, original_pred, adj_pred in zip(matches, predicted_deltas, adjusted_predicted_deltas):
        print(f"{match_data['delta_elo']} | {round(original_pred,1)} | {adj_pred}")


if __name__ == "__main__":
    main()

PONIZEJ 3:0/0:3

In [ ]:
import pymongo
import numpy as np
import time
import json

def connect_mongo_and_return_collections(login, password, IP, port, database_name, collections_name):
    """
    Nawiązuje połączenie z bazą danych MongoDB i zwraca kolekcję do operacji.
    """
    conn = pymongo.MongoClient(f'mongodb://{login}:{password}@{IP}:{port}/')
    db = conn[f'{database_name}']
    matches_coll = db[f'{collections_name}']

    try:
        # Test połączenia
        conn.admin.command('ping')
        print("Połączenie z MongoDB nawiązane!")
    except Exception as e:
        print(f"Nie udało się połączyć z MongoDB: {e}")
        exit()

    return matches_coll


def get_all_matches():
    """
    Pobiera dane z kolekcji `matches` dla wyników 3:1 lub 1:3, wykluczając określone klucze.
    """
    pipeline = [
        {
            "$match": {
                "$and": [
                    {
                        "$or": [
                            { "ft_result": "3 : 0" },
                            { "ft_result": "0 : 3" }
                        ]
                    },
                    { "delta_elo": { "$ne": 0 } }
                ]
            }
        },
        {
            "$project": {
                "_id": 0,
                "odds_url": 0,
                "home_id": 0,
                "odd_home": 0,
                "odd_away": 0
            }
        },
        {
            "$limit": 100000
        }
    ]

    matches_coll = connect_mongo_and_return_collections(
        'root', 'Name_Secret', 'IP SECRET', '27017',
        'czech_liga_pro_test', 'matches'
    )

    start_time = time.time()
    all_matches = list(matches_coll.aggregate(pipeline))
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Czas wykonania zapytania: {execution_time:.6f} sekund")

    if not all_matches:
        return "Brak wyników zapytania."
    else:
        return all_matches


def calculate_actuals_by_sets(match_data):
    """
    Oblicza wartość actual_home i actual_away dla meczu o wyniku 3:0 lub 0:3.
    """
    if int(match_data['winner']) == 1:
        actual_home = 1
    else:
        actual_home = 0

    return actual_home, 1 - actual_home


def calculate_predicted_delta_elo(match_data, K):
    """
    Oblicza przewidywaną zmianę Elo na podstawie aktualnych wartości Elo i faktycznego wyniku.
    """
    expected_home = 1 / (1 + 10 ** ((match_data['away_elo'] - match_data['home_elo']) / 108))
    expected_away = 1 - expected_home

    actual_home, actual_away = calculate_actuals_by_sets(match_data)
    
    new_home_elo = K * (actual_home - expected_home)
    new_away_elo = K * (actual_away - expected_away)

    return 1.067 * (new_home_elo), new_away_elo


def load_corrections(json_path='corrections.json'):
    """
    Wczytuje dane korekcyjne z pliku JSON.
    Oczekiwany format: { "1.3": 0.2, "1.4": 0.3, ... }
    """
    with open(json_path, 'r') as f:
        corrections = json.load(f)
    return corrections


def adjust_predicted_delta(p, corrections):
    """
    Dopasowuje przewidywaną zmianę elo (p) na podstawie wczytanych z pliku JSON korekt.

    Zasada:
    1. Zaokrąglamy p do jednej cyfry po przecinku.
    2. Konwertujemy na string, żeby dopasować do klucza w słowniku.
    3. Jeśli jest w corrections, to znaczy że mamy dla tej wartości specjalną korektę.
       W tym wypadku 'korekta' może oznaczać nową wartość predicted_delta_elo. 
       Można zinterpretować to tak, że wartość w 'corrections' to finalna korekta (np. średnia delta_elo z terminala).
    4. Jeśli klucza nie ma w corrections, zostawiamy p bez zmian (lub można zrobić inną logikę).

    Zwracamy zaokrągloną do jednej cyfry po przecinku wartość.
    """

    p_rounded = round(p, 1)
    p_str = f"{p_rounded}"

    if p_str in corrections:
        adj = corrections[p_str]
    elif abs(p) >= 15.2: 
        # to ze nie bylo takiego przypadku w bazie danych nie znaczy ze nie moze sie wydarzyc. Zabezpieczenie w razie rzadkiego meczu.
        adj = p_rounded * 1.2
    else:
        # Jeśli nie ma dopasowania w słowniku korekt, zostawiamy p_rounded bez zmian.
        adj = p_rounded

    adj = round(adj, 1)
    return adj


def calculate_mean_error(matches, adjusted_predicted_deltas):
    """
    Funkcja oblicza średni błąd pomiędzy rzeczywistą zmianą elo (delta_elo),
    a zmodyfikowaną przewidywaną zmianą elo.
    """
    mean_error = 0
    for match_data, adj_pred in zip(matches, adjusted_predicted_deltas):
        mean_error += abs(match_data['delta_elo'] - adj_pred)

    if len(matches) > 0:
        return mean_error / len(matches)
    else:
        return 0


def main():
    matches = get_all_matches()

    if isinstance(matches, str):
        print(matches)
        return

    corrections = load_corrections('corrections_3_0.json')

    predicted_deltas = []
    mean_error = 0
    print("original_delta_elo | predicted_delta_elo | predicted_delta_elo_adjusted")
    for match_data in matches:
        pred = (calculate_predicted_delta_elo(match_data, 15))[0]
        predicted_deltas.append(pred)
        mean_error += abs(match_data['delta_elo'] - pred)

    if len(matches) > 0:
        print(f"Średni błąd: {mean_error/len(matches)}")
    else:
        print("Brak danych do obliczeń.")
        return

    adjusted_predicted_deltas = [adjust_predicted_delta(p, corrections) for p in predicted_deltas]

    adjusted_mean_error = calculate_mean_error(matches, adjusted_predicted_deltas)
    print(f"Średni błąd po modyfikacji predykcji: {adjusted_mean_error}")


    for val in np.arange(0.0, 5.0, 0.1):
        delta_elo_values = []
        for match_data, original_pred, adj_pred in zip(matches, predicted_deltas, adjusted_predicted_deltas):
            # Zaokrąglamy do jednej cyfry po przecinku
            if round(original_pred, 1) == round(val, 1):
                delta_elo_values.append(match_data['delta_elo'])
        
        if len(delta_elo_values) > 0:
            avg_delta_elo = sum(delta_elo_values) / len(delta_elo_values)
            print(f"{val}: {avg_delta_elo}")
        else:
            print(f"Brak meczów o predicted_delta_elo == {val}")

    for match_data, original_pred, adj_pred in zip(matches, predicted_deltas, adjusted_predicted_deltas):
        print(f"{match_data['delta_elo']} | {round(original_pred,1)} | {adj_pred}")


if __name__ == "__main__":
    main()

Dla 3:0 mamy sr. blad bezw.: **0.0528** ELO   
Dla 3:1 mamy sr. blad bezw.: **0.0452** ELO  
Dla 3:2 mamy sr. blad bezw.: **0.0314** ELO
